In [4]:
import pandas as pd
import json
import os
import xml.etree.ElementTree as ET
import requests
import saspy
from io import BytesIO
from zipfile import ZipFile
from dateutil import parser

## Get Data

In [5]:
def read_remote_zip(url):
    response = requests.get(url, stream=True)
    zip_file = ZipFile(BytesIO(response.content))
    return zip_file
    
zip_hr       = read_remote_zip("https://github.com/stu-code/viz/raw/refs/heads/main/snowboarding/data/biometrics/hr/hr.zip")
zip_hr_var   = read_remote_zip("https://github.com/stu-code/viz/raw/refs/heads/main/snowboarding/data/biometrics/hr_variability/hr_variability.zip")
zip_spo2     = read_remote_zip("https://github.com/stu-code/viz/raw/refs/heads/main/snowboarding/data/biometrics/spo2/spo2.zip")
zip_spo2_var = read_remote_zip("https://github.com/stu-code/viz/raw/refs/heads/main/snowboarding/data/biometrics/spo2_variability/spo2_variability.zip")
zip_gps      = read_remote_zip("https://github.com/stu-code/viz/raw/refs/heads/main/snowboarding/data/gps/gps.zip")

## Read Heartrate Data

In [6]:
df_list  = []
    
for json_file in zip_hr.namelist():
    with zip_hr.open(json_file) as f:
        data = json.load(f)
        
    df = pd.json_normalize(data, sep='_')

    df.columns = (
        df.columns.str.lower()
                  .str.replace('value_', '')
                  .str.replace('datetime', 'timestamp')
    )
    
    # Convert to datetime and from UTC to Mountain Time
    df['timestamp'] = ( 
        pd.to_datetime(df['timestamp'], format='%m/%d/%y %H:%M:%S', utc=True)
          .dt.tz_convert('US/Mountain')
          .dt.tz_localize(None)
    )
    
    df_list.append(df)
        
df_hr = pd.concat(df_list, ignore_index=True)

## Read Biometric CSV Data

In [7]:
df_list = []

def read_bio_csv(zip_file):    
    for csv_file in zip_file.namelist():
        with zip_file.open(csv_file) as f:
            df = pd.read_csv(f, parse_dates=['timestamp'])
            
        df_list.append(df)
        
    return pd.concat(df_list, ignore_index=True)

df_hr_var   = read_bio_csv(zip_hr_var)
df_spo2     = read_bio_csv(zip_spo2)
df_spo2_var = read_bio_csv(zip_spo2_var)

C:\Users\stsztu\AppData\Local\Temp\ipykernel_3164\1981936932.py:6: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df = pd.read_csv(f, parse_dates=['timestamp'])
C:\Users\stsztu\AppData\Local\Temp\ipykernel_3164\1981936932.py:6: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df = pd.read_csv(f, parse_dates=['timestamp'])
C:\Users\stsztu\AppData\Local\Temp\ipykernel_3164\1981936932.py:6: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df = pd.read_csv(f, parse_dates=['timestamp'])
C:\Users\stsztu\AppData\Local\Temp\ipykernel_3164\1981936932.py:6: UserWarning: Could not infer form

## Read GPS Data

In [8]:
''' Read GPS data in GPX format without needing to import a separate GPX 
    package. GPX data looks like this:
        
    <?xml version="1.0" encoding="UTF-8"?>
    <gpx xmlns="http://www.topografix.com/GPX/1/1" xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance" xmlns:gte="http://www.gpstrackeditor.com/xmlschemas/General/1" xsi:schemaLocation="http://www.topografix.com/GPX/1/1 http://www.topografix.com/GPX/1/1/gpx.xsd" version="1.1" creator="Slopes for Android - http://getslopes.com">
      <trk>
        <name>Jan 25, 2024 - Keystone Resort</name>
        <trkseg>
          <trkpt lat="39.605675" lon="-105.941414">
            <ele>2856.891977</ele>
            <time>2024-01-25T09:13:52.453-07:00</time>
            <hdop>19</hdop>
            <vdop>4</vdop>
            <extensions>
              <gte:gps speed="1.317580" azimuth="212.300003"/>
            </extensions>
          </trkpt>
       </trkseg>
      </trk>
    </gpx>
    
    There are two namespaces we need to use:
        1. The gpx namespace: http://www.topografix.com/GPX/1/1
        2. The gte namespace http://www.gpstrackeditor.com/xmlschemas/General/1
        
    The gte namespace is used to extract gps and azimuth data from the 
    <extensions> tag
'''
gpx_namespace = '{http://www.topografix.com/GPX/1/1}'
gte_namespace = '{http://www.gpstrackeditor.com/xmlschemas/General/1}'
    
clean_data = []
file_list  = [file_name for file_name in zip_gps.namelist() if file_name.endswith(".gpx")]
    
for gpx_file in file_list:
    with zip_gps.open(gpx_file) as f:
        raw_data = f.read()
            
    # ET.parse expects an actual file, so BytesIO makes it behave like a file
    root = ET.parse(BytesIO(raw_data))
        
    for trkpt in root.findall(f'.//{gpx_namespace}trkpt'):
        row = {
                "timestamp":  parser.parse(trkpt.find(f'{gpx_namespace}time').text, ignoretz=True),
                "lat":       float(trkpt.get("lat")),
                "lon":       float(trkpt.get("lon")),
                "elevation": float(trkpt.find(f'{gpx_namespace}ele').text),
                "speed":     float(trkpt.find(f'.//{gpx_namespace}extensions/{gte_namespace}gps').get("speed")),
                "azimuth":   float(trkpt.find(f'.//{gpx_namespace}extensions/{gte_namespace}gps').get("azimuth"))
              }
        
        clean_data.append(row)

df_gps = pd.DataFrame(clean_data)

## Read GPS Metadata

In [9]:
# Read in GPS metadata. This will help us more easily define runs and lifts
# and also give us some additonal information if we want to use it
   
df_list   = []
file_list = [file_name for file_name in zip_gps.namelist() if file_name.endswith(".slopes")]
    
# .slopes files are just zip files with some CSVs and XML metadata.
# We just want to read Metadata.xml
for slopes_file in file_list:
    with zip_gps.open(slopes_file) as f:
        raw_data = f.read()

    with ZipFile(BytesIO(raw_data), 'r') as zip_file:
        with zip_file.open('Metadata.xml') as xml_file:
            df = pd.read_xml(xml_file, parser='etree', xpath='.//Action')
            
    # Convert start/end to datetimes without the timezone
    df[['start', 'end']] = df[['start', 'end']].map(lambda x: parser.parse(x, ignoretz=True))

    # Get datepart (for later processing)
    df['date'] = df['start'].dt.date
    df_list.append(df)
        
# Final GPS metadata dataframe 
df_gps_meta = pd.concat(df_list, ignore_index=True)

In [11]:
access_token = open(r"C:\Users\stsztu\casconfig\access_token.txt", 'r')

sas = saspy.SASsession(
    cfgname='httpsviya',  
    url="https://create.demo.sas.com",
    authtoken=access_token.read(), 
    cafile=r"C:\Users\stsztu\casconfig\demo-rootCA-Intermidiates_4CLI.pem"
)

SAS server started using Context SAS Studio compute context with SESSION_ID=2a670fe4-20e9-4455-89d2-7793ac2370d7-ses0000


In [12]:
sas.df2sd(df_hr, 'heartrate')
sas.df2sd(df_hr_var, 'heartrate_variability')
sas.df2sd(df_spo2, 'spo2')
sas.df2sd(df_spo2, 'spo2_variability')
sas.df2sd(df_gps, 'gps')
sas.df2sd(df_gps_meta, 'gps_meta')

Libref  = WORK
Table   = gps_meta
Dsopts  = {}
Results = Pandas

In [18]:
sas.submit(code=
'''

data heartrate;
    set heartrate;

    /* Convert to MT */
    if('25JAN2024'd LE datepart(timestamp) LE '28JAN2024'd)
        then timestamp = intnx('hour', timestamp, -7, 'S')
    ;

    /* Convert to ET */
    else if('23FEB2024'd LE datepart(timestamp) LE '24FEB2024'd)
        then timestamp = intnx('hour', timestamp, -5, 'S')
    ;
run;

'''
)

{'LOG': "337707  ods listing close;ods html5 (id=saspy_internal) options(bitmap_mode='inline') device=svg style=HTMLBlue; ods graphics on /\n337707! outputfmt=png;\nNOTE: Writing HTML5(SASPY_INTERNAL) Body file: sashtml2.htm\n337708  \n337709  \n337710  \n337711  data heartrate;\n337712      set heartrate;\n337713  \n337714      /* Convert to MT */\n337715      if('25JAN2024'd LE datepart(timestamp) LE '28JAN2024'd)\n337716          then timestamp = intnx('hour', timestamp, -7, 'S')\n337717      ;\n337718  \n337719      /* Convert to ET */\n337720      else if('23FEB2024'd LE datepart(timestamp) LE '24FEB2024'd)\n337721          then timestamp = intnx('hour', timestamp, -5, 'S')\n337722      ;\n337723  run;\n\nNOTE: There were 59429 observations read from the data set WORK.HEARTRATE.\nNOTE: The data set WORK.HEARTRATE has 59429 observations and 3 variables.\nNOTE: DATA statement used (Total process time):\n      real time           0.01 seconds\n      cpu time            0.02 seconds\n

In [20]:
sas.submit(loglines=True, code=
'''

data gps_meta;
    set gps_meta;
    date = datepart(start);

    format date date9.;
run;

proc sort data=gps_meta;
    by date type start;
run;

/* Identify run numbers */
data gps_meta;
    set gps_meta;
    by date type start;

    if(first.type) then call missing(nbr);
        
    nbr+1;
           
    drop date;
run;

'''
)

{'LOG': [{'line': "337758  ods listing close;ods html5 (id=saspy_internal) options(bitmap_mode='inline') device=svg style=HTMLBlue; ods graphics on /",
   'type': 'source',
   'version': 1},
  {'line': '337758! outputfmt=png;', 'type': 'source', 'version': 1},
  {'line': 'NOTE: Writing HTML5(SASPY_INTERNAL) Body file: sashtml4.htm',
   'type': 'note',
   'version': 1},
  {'line': '337759  ', 'type': 'source', 'version': 1},
  {'line': '337760  ', 'type': 'source', 'version': 1},
  {'line': '337761  ', 'type': 'source', 'version': 1},
  {'line': '337762  data gps_meta;', 'type': 'source', 'version': 1},
  {'line': '337763      set gps_meta;', 'type': 'source', 'version': 1},
  {'line': '337764      date = datepart(start);',
   'type': 'source',
   'version': 1},
  {'line': '337765  ', 'type': 'source', 'version': 1},
  {'line': '337766      format date date9.;', 'type': 'source', 'version': 1},
  {'line': '337767  run;', 'type': 'source', 'version': 1},
  {'line': '', 'type': 'note', 'v

In [21]:
sas.submit(code=
'''

/* Fuzzy merge GPS with heartrate data */
proc sql;
    create table _snowboarding_gps_hr(drop=dif) as
        select round(gps.timestamp) as timestamp format=datetime.2
             , datepart(gps.timestamp) as date format=date9.
             , gps.lat
             , gps.lon
             , round(gps.elevation)*3.28084 as elevation
             , round(gps.speed, .1) as speed
             , hr.bpm
             , hr.confidence as hr_sensor_confidence
             , abs(round(hr.timestamp) - round(gps.timestamp)) as dif
        from gps as gps
        LEFT JOIN
             heartrate as hr
        ON   dhms(datepart(gps.timestamp), hour(gps.timestamp), minute(gps.timestamp), 0)
           = dhms(datepart(hr.timestamp), hour(hr.timestamp), minute(hr.timestamp), 0)
        where    timepart(hr.timestamp) BETWEEN '8:30't AND '16:00't
              OR datepart(gps.timestamp) IN ('27JAN2024'd, '28JAN2024'd)
        group by calculated timestamp
        having dif = min(dif)
    ;
quit;

'''
)

{'LOG': "337788  ods listing close;ods html5 (id=saspy_internal) options(bitmap_mode='inline') device=svg style=HTMLBlue; ods graphics on /\n337788! outputfmt=png;\nNOTE: Writing HTML5(SASPY_INTERNAL) Body file: sashtml5.htm\n337789  \n337790  \n337791  \n337792  /* Fuzzy merge GPS with heartrate data */\n337793  proc sql;\n337794      create table _snowboarding_gps_hr(drop=dif) as\n337795          select round(gps.timestamp) as timestamp format=datetime.2\n337796               , datepart(gps.timestamp) as date format=date9.\n337797               , gps.lat\n337798               , gps.lon\n337799               , round(gps.elevation)*3.28084 as elevation\n337800               , round(gps.speed, .1) as speed\n337801               , hr.bpm\n337802               , hr.confidence as hr_sensor_confidence\n337803               , abs(round(hr.timestamp) - round(gps.timestamp)) as dif\n337804          from gps as gps\n337805          LEFT JOIN\n337806               heartrate as hr\n337807        

In [22]:
sas.submit(code=
'''

/* Add in types: lift or run */
proc sql;
    create table snowboarding_gps_hr as
        select gps.*
               , meta.type
               , CASE(meta.type)
                     when('Lift') then meta.nbr
                     else .
                 END as lift_nbr
               , CASE(meta.type)
                     when('Run') then meta.nbr
                     else .
                 END as run_nbr
        from _snowboarding_gps_hr as gps
        LEFT JOIN
             gps_meta as meta
        ON gps.timestamp BETWEEN meta.start AND meta.end
        order by gps.timestamp
    ;
quit;

/* De-dupe timestamps */
proc sort data=snowboarding_gps_hr nodupkey;
    by timestamp;
run;

'''
)

{'LOG': "337819  ods listing close;ods html5 (id=saspy_internal) options(bitmap_mode='inline') device=svg style=HTMLBlue; ods graphics on /\n337819! outputfmt=png;\nNOTE: Writing HTML5(SASPY_INTERNAL) Body file: sashtml6.htm\n337820  \n337821  \n337822  \n337823  /* Add in types: lift or run */\n337824  proc sql;\n337825      create table snowboarding_gps_hr as\n337826          select gps.*\n337827                 , meta.type\n337828                 , CASE(meta.type)\n337829                       when('Lift') then meta.nbr\n337830                       else .\n337831                   END as lift_nbr\n337832                 , CASE(meta.type)\n337833                       when('Run') then meta.nbr\n337834                       else .\n337835                   END as run_nbr\n337836          from _snowboarding_gps_hr as gps\n337837          LEFT JOIN\n337838               gps_meta as meta\n337839          ON gps.timestamp BETWEEN meta.start AND meta.end\n337840          order by gps.time

In [23]:
sas.submit(code=
'''

cas;
caslib _ALL_ assign;

/* Load, promote and save to CAS */
proc casutil incaslib='casuser' outcaslib='casuser';
    droptable casdata='snowboarding_gps_hr' quiet;
    load data=snowboarding_gps_hr promote;
    droptable casdata='gps_meta' quiet;
    load data=gps_meta promote;

    save casdata='snowboarding_gps_hr' replace;
    save casdata='gps_meta' replace;
quit;

'''
)

{'LOG': '337852  ods listing close;ods html5 (id=saspy_internal) options(bitmap_mode=\'inline\') device=svg style=HTMLBlue; ods graphics on /\n337852! outputfmt=png;\nNOTE: Writing HTML5(SASPY_INTERNAL) Body file: sashtml7.htm\n337853  \n337854  \n337855  \n337856  cas;\nNOTE: The session name identified with the SESSREF= SAS option is connected to Cloud Analytic Services. The default value for \n      SESSREF= is CASAUTO.\n337857  caslib _ALL_ assign;\nNOTE: A SAS Library associated with a caslib can only reference library member names that conform to SAS Library naming conventions.\nNOTE: CASLIB AIoTPgMeta for session CASAUTO will not be mapped to SAS Library AIoTPgMeta. The CASLIB name is not valid for use as a \n      libref.\nNOTE: CASLIB ALC for session CASAUTO will be mapped to SAS Library ALC.\nNOTE: CASLIB AO_REF for session CASAUTO will be mapped to SAS Library AO_REF.\nNOTE: CASLIB AO_RES for session CASAUTO will be mapped to SAS Library AO_RES.\nNOTE: CASLIB CASUSER(Stu.Szt